In [39]:
import random as rad
#Khởi tạo mạng neuron
def init_network(n_input, n_hidden, n_output):
    network = list()
    #Trọng số của mỗi đầu vào n_input và bias đến lớp hidden
    hidden_layer = [{'weights':[rad.random() for i in range(n_input + 1)]} for i in range(n_hidden)]
    network.append(hidden_layer)
    #Trọng số của mỗi đầu vào n_hidden và bias đến lớp output
    output_layer = [{'weights':[rad.random() for j in range(n_hidden + 1)]} for j in range(n_output)]
    network.append(output_layer)
    return network
network = init_network(2,1,2)
print(network)

[[{'weights': [0.6351308271473181, 0.778944915444524, 0.610968655957299]}], [{'weights': [0.9018410047992774, 0.021626194304731783]}, {'weights': [0.2809829135500509, 0.5684408405128863]}]]


In [40]:
#Tính hàm kích hoạt cho mỗi đầu vào
def activation_input(weights, inputs):
    bias = weights[-1]
    activation = bias
    for i in range(len(weights)-1):
        activation += weights[i]*inputs[i]
    return activation

In [41]:
#Tính hàm sigmoid
import math as mat
def sigmoid(activation):
    return 1.0/(1.0 + mat.exp(-activation))

In [42]:
#Tính lan truyền thẳng từ input đến output
#Forward Propagation
def forward_pro(network, data_row):
    inp = data_row
    for layer in network:
        temp = []
        for neuron in layer:
            activation = activation_input(neuron['weights'], inp)
            neuron['output'] = sigmoid(activation)
            temp.append(neuron['output'])
            print(neuron)
        inp = temp
    return inp
print(forward_pro(network, [1,0]))

{'weights': [0.6351308271473181, 0.778944915444524, 0.610968655957299], 'output': 0.7766239327925357}
{'weights': [0.9018410047992774, 0.021626194304731783], 'output': 0.6730511300017334}
{'weights': [0.2809829135500509, 0.5684408405128863], 'output': 0.6871134809115805}
[0.6730511300017334, 0.6871134809115805]


In [43]:
#Tính đạo hàm của hàm sigmoid ở đầu ra 
def sigmoid_derivative(output):
    return output*(1.0-output)

In [44]:
#Tính hàm lỗi của backpropagation và lưu trong network
def backpropagation_error(network, expection):
    for j in reversed(range(len(network))):
        layer = network[j]
        errs = list()
        if j != len(network) - 1:
            for k in range(len(layer)):
                err = 0.0
                for neuron in network[j+1]:
                    err += (neuron['weights'][j]*neuron['error'])
                errs.append(err)
        else:
            for k in range(len(layer)):
                neuron = layer[k]
                errs.append(expection[k] - neuron['output'])
        for k in range(len(layer)):
            neuron = layer[k]
            neuron['error'] = errs[j] * sigmoid_derivative(neuron['output'])
network = [[{'output': 0.7105668883115941, 'weights': [0.9463936036227082, 0.6431101871813475, 0.4264771599829553]}],[{'output': 0.6213859615555266, 'weights': [0.9276732185160963, 0.6167567609526338]}, {'output': 0.6573693455986976, 'weights': [0.6731316543638922, 0.6552006657715309]}]]    
expection = [0, 1]
backpropagation_error(network,expection)
for layer in network:
    print(layer)

[{'output': 0.7105668883115941, 'weights': [0.9463936036227082, 0.6431101871813475, 0.4264771599829553], 'error': 0.02606269557370685}]
[{'output': 0.6213859615555266, 'weights': [0.9276732185160963, 0.6167567609526338], 'error': 0.08060915452180441}, {'output': 0.6573693455986976, 'weights': [0.6731316543638922, 0.6552006657715309], 'error': 0.0771723774346327}]


In [48]:
def update_weights(network, row, l_rate):
    for i in range(len(network)):
        inputs = row[:-1]
        if i != 0:
            inputs = [neuron['output'] for neuron in network[i - 1]]
        for neuron in network[i]:
            for j in range(len(inputs)):
                neuron['weights'][j] += l_rate * neuron['error'] * inputs[j]
            neuron['weights'][-1] += l_rate * neuron['error']

In [49]:
def train_network(network, train, l_rate, n_epoch, n_outputs):
    for epoch in range(n_epoch):
        sum_error = 0
        for row in train:
            outputs = forward_pro(network, row)
            expected = [0 for i in range(n_outputs)]
            expected[row[-1]] = 1
            sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))])
            backpropagation_error(network, expected)
            update_weights(network, row, l_rate)
        print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))

In [50]:
dataset = [[2.7810836,2.550537003,0],
            [1.465489372,2.362125076,0],
            [3.396561688,4.400293529,0],
            [1.38807019,1.850220317,0],
            [3.06407232,3.005305973,0],
            [7.627531214,2.759262235,1],
            [5.332441248,2.088626775,1],
            [6.922596716,1.77106367,1],
            [8.675418651,-0.242068655,1],
            [7.673756466,3.508563011,1]]
n_inputs = len(dataset[0]) - 1
n_outputs = len(set([row[-1] for row in dataset]))
network = init_network(n_inputs, 2, n_outputs)
train_network(network, dataset, 0.5, 20, n_outputs)
for layer in network:
    print(layer)

{'weights': [0.3476332396365638, 0.4486202065680013, 0.8300165620747163], 'output': 0.9498381155403307}
{'weights': [0.6333260028649264, 0.561343611182555, 0.1159582479046185], 'output': 0.9647369044018836}
{'weights': [0.7038877884627419, 0.4180049877168144, 0.8274262287142131], 'output': 0.8698089402967978}
{'weights': [0.8572688858613496, 0.17623018611554564, 0.3582691965035595], 'output': 0.7929092418753569}
{'weights': [0.3360509894564935, 0.4379981031323934, 0.825851908347633], 'output': 0.9131674219094842, 'error': -0.008329307454166634}
{'weights': [0.6250561300303601, 0.5537592952437369, 0.11298463257188768], 'output': 0.9118990892316083, 'error': -0.0059472306654616645}
{'weights': [0.6612447534853512, 0.37469307078559455, 0.7825311731466309], 'output': 0.8491616211623594, 'error': -0.08979011113516447}
{'weights': [0.7954348953155568, 0.11342629179927194, 0.2931696921415793], 'output': 0.7545388824643248, 'error': -0.1301990087239604}
{'weights': [0.3258793602727549, 0.42160